In [1]:
import tensorflow as tf
from tensorflow import keras

In [3]:
import keras_tuner as kt


Using TensorFlow backend


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Download and prepare the dataset
In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the Fashion MNIST dataset.

In [4]:
#load data
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
#normalise pixel values between 0 and 1

img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# Define the model
When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.

You can define a hypermodel through two approaches:

By using a model builder function
By subclassing the HyperModel class of the Keras Tuner API

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 34s]
val_accuracy: 0.8654999732971191

Best val_accuracy So Far: 0.8951666951179504
Total elapsed time: 00h 06m 25s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.001.



# Train the model
Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.4956 - accuracy: 0.8238 - val_loss: 0.4433 - val_accuracy: 0.8393
Epoch 2/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3663 - accuracy: 0.8676 - val_loss: 0.3711 - val_accuracy: 0.8655
Epoch 3/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3295 - accuracy: 0.8797 - val_loss: 0.3321 - val_accuracy: 0.8826
Epoch 4/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3062 - accuracy: 0.8857 - val_loss: 0.3474 - val_accuracy: 0.8765
Epoch 5/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2850 - accuracy: 0.8950 - val_loss: 0.3278 - val_accuracy: 0.8829
Epoch 6/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2695 - accuracy: 0.9000 - val_loss: 0.3453 - val_accuracy: 0.8763
Epoch 7/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2550 - accuracy: 0.9065 - val_loss: 0.3193 - val_accuracy:

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/33
1500/1500 [==============================] - 4s 2ms/step - loss: 0.4920 - accuracy: 0.8260 - val_loss: 0.3861 - val_accuracy: 0.8662
Epoch 2/33
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3695 - accuracy: 0.8649 - val_loss: 0.3608 - val_accuracy: 0.8687
Epoch 3/33
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3301 - accuracy: 0.8774 - val_loss: 0.3456 - val_accuracy: 0.8764
Epoch 4/33
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3051 - accuracy: 0.8876 - val_loss: 0.3349 - val_accuracy: 0.8783
Epoch 5/33
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2867 - accuracy: 0.8928 - val_loss: 0.3267 - val_accuracy: 0.8824
Epoch 6/33
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2711 - accuracy: 0.8993 - val_loss: 0.3194 - val_accuracy: 0.8852
Epoch 7/33
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2569 - accuracy: 0.9031 - val_loss: 0.3381 - val_accuracy:

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 0s 912us/step - loss: 0.4675 - accuracy: 0.8839
[test loss, test accuracy]: [0.467483788728714, 0.883899986743927]
